# COGS 118A - Final Project

# Recognizing Tiles of Online Chessboards

## Group members

- Cameron VanderTuig
- Zeven Vidmar Barker
- Nawwar Tohme
- Nicholas DeGroot

# Abstract 
Our project aims to take in an image of a chess board and output where every piece lies on the board, in order to produce a board state which is usable by a computer analysis software. To do so, we will use the [`koryakinp/chess-positions`](https://www.kaggle.com/datasets/koryakinp/chess-positions) Kaggle dataset made up of 100,000 images of Chess boards in various styles along with the true positions of every piece. Our project will then compare multiple machine learning models learned in class (SVM, Logistic Regression, etc.) to determine the best model by percentage of board correctly identified.

We believe this work will have profound effects on the Chess community, who will be able to quickly digitize and transform their games into various formats and perform more complex analysis of their play styles with the help of AI Chess bots. 

# Background

The world of Online Chess is home to a plethora of different platforms and tools that allow players to both play Chess with others and watch these games being streamed in real-time, each with their own respective styles and quality. While Chess is no stranger to organized notation for classifying pieces and movements <a name="wikipedia"></a>[<sup>[1]</sup>](#notation), the variety of online formats and mediums may produce problems or limitations when it comes to scraping this information directly from websites.

Visualizing and classifying chess pieces and chess boards is a higher-level problem that extends to physical chessboards, namely in deep learning algorithms such as LiveChess2FEN <a name="livechess"></a>[<sup>[2]</sup>](#livechess). For our expertise level, we will stick to the varying art styles seen across different Chess game softwares and websites, focusing to tackle the online mediums primarily adopted by game streamers and digital content creators.

Something similar to this has been done by chessvision.ai, which does essentially what we are aiming to do, and enables scanning an image of a chess layout in order to produce a usable board state.

# Problem Statement

Chess is one of the most popular games of the 21st century, with millions of people around the world competing to see who can checkmate their opponent. The advent of AI Chess bots allowed people to quickly analyze their games against top-level players to see exactly where they went wrong and improve their play at a rate unseen in history.

The limiting factor, however, was that players needed to represent their games digitally for the first time. This can take a lot of work, especially considering how fast the game can progress across several moves.

This is where our project comes in. We hope to produce a model that can take in a simple image of a Chess board and output a standardized format that anyone can plug into their favorite Chess trainer of choice. We expect this to rapidly improve the rate at which people can learn, thus forwarding the game of Chess as a whole.


# Data

The dataset we're using comes from Kaggle, under [`koryakinp/chess-positions`](https://www.kaggle.com/datasets/koryakinp/chess-positions).

The entire dataset consists of 100,000 RGB images in `.jpeg` format, pre-split into 80,000 train and 20,000 test sets. Data is formatted as follows:
- The name of the image is the board state in Forsyth-Eswards (FEN) format. You can read more about that [here](https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation).
  - Our model uses a FEN parser to clean this format into a sparser 8x8 matrix with each cell representing the piece.
- Each image is 400x400 pixels, each with a RGB value. resulting in 64,000,000 x 3 = 192,000,000 total observations per data point.
  - Our models will be concerned with individual pieces. Taking this into account subdivides the image into an 8x8 grid, with each piece representing a 50x50 slice of the image.
    - This results in 2,500 x 3 = 7,500 variables
    - We grayscale each subplot (back to 2,500 variables), since the essence of the feature is still represented quite well.

During data cleaning, we found that cleaning the entire dataset quickly filled our hard drives to well over 100GBs. For this reason, we chose to limit the number of observations to the first 2,500 training images and 500 test images. This sorted out to a more managable 2GB of space! 


# Proposed Solution

Our project will compare the effectiveness of various machine learning problems, mainly focusing on multi-class image classification.

To do so, we will split the chess board into its 64 pieces using standard image processing. One possible implementation of this comes from SciKit-Image as view_as_windows (https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.view_as_windows). Depending on the model and its limitations, we may also grayscale the image. Each square of the board will then be run through the model for 13-way classification: BLACK/WHITE variations of PAWN, ROOK, KNIGHT, BISHOP, KING, and QUEEN, plus an EMPTY classification.

We expect to test the following models, likely using their Scikit-learn implementations:
- SVM
- Logistic Regression
- Ridge Regression
- K-Nearest Neighbors
- Decision Trees

If time allows, we may also attempt to train and test a Convolutional Network model using PyTorch. This gives us the opportunity to not need to pre-split the image into separate board positions and instead do it all at once.

Evaluation will be assessed via model accuracy on the number of spaces correctly classified.


# Evaluation Metrics

For this project, we decided to focus on the **percentage of correctly identified cells**. That is:
$$
\frac{\text{Number of Cells Correctly Identified}}{\text{Number of Cells Total}}
$$


There were a number of points that went into our decision.
- **It’s easy to interpret**: especially compared to other metrics, cell accuracy is much clearer and simpler to understand.
- **It allowed equal comparisons between models**: we didn’t want to handicap our model selection to just those that produced scores. Using accuracy only takes into account the final prediction of the model, regardless of how it actually got there.
- **It makes sense for Chess**: one of the main arguments our group had was whether our metric should weight the classes. For example, is it more important that a piece is missing (incorrectly identified as EMPTY class) or added (empty class incorrectly identified as something else)? Based on our (albeit limited) experience with chess, we decided that the two errors were of relative equal importance and that we could treat them as such.
- **It's achievable**: We also considered working with **percentage of correctly identified _boards_**, however dropped that idea when faced with how hard the problem ended up being.

We also included a post-hoc analysis of our model predictions via a confusion matrix to see where exactly the model was going wrong. This allowed us to ensure our m

# Results

### Subsection 1: Preprocessing

At the start of our project, we first needed to decide how to represent our data in a format understood by scikit-learn. After all, scikit speaks the language of vectors and matrices - all we were given were strings and images. Thankfully, we were in good hands when it came to images. Scikit-learn has a sister project similarly named scikit-image, which supports various image handling operations. Of particular interest to this project was reading an image from memory, grayscaling, and reshaping the image to set-size blocks. This allowed us to turn our downloaded chess boards into 8x8x50 tensors, representing the rows/columns/pixels of each chess cell respectively.

When attempting to preproccess all the dataset’s images, we quickly found out why image compression exists. These matrices almost completely filled up our hard drives, even with pickle compression enabled. This forced us to only consider 3k boards, which we subsequently split into a 2.5k training set and 500 test set. Even this reduced size still took up almost 2GB of data!

Labels were slightly more complicated to figure out. We first attempted to utalize a popular Python library under the name python-chess, yet ended up deep diving into FEN notation to craft a custom solution. 

#### A Brief Explanation of FEN Notation
We’ll use 1b1B1Qr1-7p-6r1-2P5-4Rk2-1K6-4B3-8 as an example. Here’s that in image form.

![](./figures/chess_board.jpg)

FEN notation is split up into 8 sections, each representing a different row. Normally this is done through the / character, yet due to file naming restrictions our dataset replaced it with the - character. For our example, we can break it into the following sections.
- 1b1B1Qr1
- 7p
- 6r1
- 2P5
- 4Rk2
- 1K6
- 4B3
- 8

Each row can subsequently be read through the following algorithm:
Read the next character
1. If it’s a NUMBER, place that many blank spaces
1. If it’s a K, place a King
1. If it’s a Q, place a Queen
1. If it’s a R, place a Rook
1. If it’s a B, place a Bishop
1. If it’s a N, place a Knight
1. If it’s a P, place a Pawn
1. The piece’s color is determined by whether it’s upper or lower case
     - Upper = white
     - Lower = black

So, for our example,
- 1b1B1Qr1 = SPACE - BLACK BISHOP - SPACE - WHITE BISHOP - SPACE - WHITE QUEEN - BLACK ROOK - SPACE
- 7p = SPACE - SPACE - SPACE - SPACE - SPACE - SPACE - SPACE - BLACK PAWN
- Etc.

Here’s that same algorithm in code:
```python
def parse_fen(fen: str):
    def parseRow(fenRow: str):
        row = []
        for char in fenRow:
            if char.isdigit():
                row += ['blank'] * int(char)
            else:
                row.append(char)
        return row
 
    board = [parseRow(row) for row in fen.split('-')]
    return np.array(board)
```

### Subsection 2: The Baseline Model

We started as all good machine learning projects go by developing a minimal effort baseline model that we could base all subsequent models off of. In the context of our chess boards, we thought a model that always predicts the most popular class would serve this role well (particularly considering the imbalanced class distribution towards blank spaces).

This simple model correctly predicted 83.95% of all board spaces! The confusion matrix for this can be seen below.

![](./figures/dummy.png)

### Subsection 3: Scikit-Learn Models

#### Logistic Regression
During hyperparameter testing of the sk-learn `LogisticRegression` model, we found that the `saga` solver was necessary for a dataset of our size; with the default `lbfgs`, it was failing to converge for a number of max iteration attempts. With this `saga` solver in mind, the main hyperparameters we tested were the penalty terms (`l1` and `l2`, since `elasticnet` was throwing errors), several C values from 0.01 to 10, and the class weights (due to the sheer imbalance of our dataset). 
With the default `balanced` class_weight method, we found that it was not very compatible with our dataset (the reason has yet to be discovered) – producing accuracy scores of ~0.03. Otherwise, the model was consistently predicting “empty” tiles, resulting in an equivalent accuracy of 0.828 – though still below baseline. Ultimately, the remaining hyperparameters didn’t produce promising results, though `l2` did outperform `l1` to some degree.

![](./figures/logistic.png)

As seen above, `class_weights=balanced` appears to overcompensate for the empty tile to nonempty tile ratio.

#### Support Vector Machine
When fitting an SVM model, when testing various hyperparameters, it seemed that the model would default to guessing an empty space, regardless of what is input. This resulted in a score of about the same as the baseline model at 83.2% with a linear kernel and a C value of .0001.

![](./figures/svm.png)

#### K Nearest Neighbors & Decision Trees
The K Nearest Neighbor and Decision Tree algorithms both had the same problem as the previous models. No matter which parameters were tuned, the algorithm put a substantial emphasis on guessing the baseline model of only blank spaces. When used with balanced class weights, it never predicted the blank space, and the score dropped substantially. Interestingly, in this case, the King was predicted substantially more than the other options even though there wasn’t a substantial amount of kings over other pieces (besides blank) in the dataset.

*KNN Confusion Matrix*

![](./figures/knn.png)

*Decision Tree (Balanced Class Weights)*

![](./figures/decision_balanced.png)

*Decision Tree (No Class Weights)*

![](./figures/decision_no.png)

### Subsection 4: Convolutional Neural Networks
While scikit-learn does a great job at simplifying the machine learning workflow into a compact and easy to understand format, there are several neural network algorithms that it hasn’t yet implemented. Of particular interest to our problem is Convolutional Neural Networks, which work by stacking different convolution and pooling layers one after the other to form a powerful non-linear model. Their success in computer vision has exploded in recent years and have been implemented in many of the top ImageNet models <a name="imagenet"></a>[<sup>[3]</sup>](#imagenet).

To take advantage of this algorithm, we decided to create a custom PyTorch model. The model consisted of four layers (three convolution and one max pool).

```
  | Name      | Type             | Params
-----------------------------------------------
0 | cnn_1     | Conv2d           | 130
1 | cnn_2     | Conv2d           | 168
2 | pool_1    | MaxPool2d        | 0
3 | cnn_3     | Conv2d           | 117
```

Due to the differing underlying architectures (numpy vs PyTorch), we needed to rewrite our data cleaning pipelines in this new format. Thankfully, this wasn’t too bad with the help of some new PyTorch libraries.

By specifically choosing specific kernel sizes, we were able to parallalize matrix operations across the entire image. Each kernel was crafted to only consider numbers within each cell of the board - allowing weights to be shared across cells but outputs to remain distinct.
```python
self.cnn_1 = nn.Conv2d(1, 5, (5, 5), 5) # Output: 5x80x80 (cells: 5x10x10)
self.cnn_2 = nn.Conv2d(5, 8, (2, 2), 2) # Output: 8x40x40 (cells: 8x5x5)
self.pool_1 = nn.MaxPool2d((5, 5), 5) # Output: 8x8x8 (cells: 8x1x1)
self.cnn_3 = nn.Conv2d(8, 13, (1, 1)) # Output: 12x8x8 (cells: 13x1x1)
```

While such a model could have made use of surrounding cells as extra information, we decided that such information would be extremely hard for the model to learn (essentially needing to learn how pieces interact).

Training the model involved running each chess board through the model, calculating the CrossEntropy between the predicted board state and the actual board state, backpropagating the result through the model to get the gradients, then changing the weights accordingly. Thankfully, PyTorch simplifies all this down to a simple `loss = self.criterion(scores, y)`!

After training for roughly 10min on a GPU accelerated device, we achieved a test accuracy of 98.08%. An overview of the training process and resulting confusion matrix can be found in the below as validation accuracy and validation loss curves. Note that unlike the confusion matrix's prior, this has been normalized for each TRUE row to make it easier to pick out where the model fails.

_Val Acc_

<img src="./figures/val_acc.svg" alt="Val Acc" width="300"/>

_Val Loss_

<img src="./figures/val_loss.svg" alt="Val Loss" width="300"/>

![](./figures/cnn_grayscale_confusion.png)

We also tested to see if supplying the model with RGB values (instead of our original grayscale image) made a difference. After training for roughly 15min, we achieved a test accuracy of 97.67%. The confusion matrix can be seen below.

![](./figures/cnn_rgb_confusion.png)

### Interpreting the result

#### Scikit Models Did Not Work Well
The main takeaway we ended with was Scikit models’ inability to model the data. No matter what model or hyperparameters we through at the model, nothing seemed to be able to 

Left unweighted, each model tended to always predict the empty class.
Balanced class weights NEVER predicted the empty class
Custom class weights weren’t able to capture what we wanted

#### Convolutional Neural Networks are Successful for a Reason
While state-of-the-art convolutional neural networks have upwards of 2.3 billion parameters, we were able to achieve near perfect accuracy on our problem with <1K parameters. This far outperformed anything our simpler Scikit-learn models were able to do, but required significantly more work to pull off.

Future work on democratizating these neural networks architectures to the level of scikit-learn may prove extremely useful to the AI field as a whole!
#### Data Cleaning Takes a Village
One of the most underappreciated aspects of machine learning is all the data cleaning that goes on before you can even touch a model. Translating between formats, normalizing values, image transformations, the list goes on and on. We severely underestimated just how much time this would take, but making sure it was done right made our model training much easier and more efficient.
#### Image Compression is Severely Underappreciated
Transforming all the chess images to their pure matrix form resulted in HUGE file sizes. With how many images are shared around the internet every day, this really adds up. Prior to this project we had no idea just how vital image compression is to the modern state of the web.

### Limitations

With the way that the data was generated, we didn’t have access to the metadata behind the chess board style. We only had access to the FEN notation for the board state and the image pixels of that board state. If we had that data available, we could have explored more topics, such as estimating the board style from the pieces, quantifying which board styles gave our algorithms the most trouble, etc. We also don’t have access to other metadata about the actual state of chess play, such as which player’s turn it is. Having such information could open avenues to other predictive work.

Many of our algorithms had a really hard time getting any real work done. When we didn’t weigh the classes based on their relative frequency, our models consistently predicted only empty board pieces, or nearly only empty board pieces. When we weighted each class inversely proportionally to its abundance in the dataset, we got terrible accuracy, in the 2-3% range, and even then the proportion of predicted non-empty chess pieces to truth was orders of magnitude different. 

Since we had so much data, training was extraordinarily long. Upwards of hours of training was needed to get through all the data for our simplest algorithms, for not nearly any gain in accuracy. This was especially debilitating when trying to perform hyperparameter tuning, as the iterations can easily multiply up to numbers unfeasible. 

Critically, though, for our simpler models that weren’t neural networks, we would need to find a way to solve the issue of gravitating towards empty spaces whilst still being able to achieve a high accuracy (at least higher than just guessing empty space every time). We are not sure if this type of computer vision problem is reasonable with the simple algorithms we first chose.

### Ethics & Privacy

While the prospect of classifying chess pieces across different visual styles doesn't immediately bring to mind any data privacy concerns (as these chess piece and chess board images are not unique to any particular user), issues may potentially arise in the applications of an algorithm which can identify pieces and positions on a chess board. If such an algorithm is to be used on mediums where the underlying data behind a chess game is not retrievable, such as a video replay or online livestream, issues may arise if these games are represented and analyzed when the original user doesn't intend for them to be. Some content creators, for instance, might want credit for their gameplay, but such an algorithm could potentially allow users to port and fabricate gameplay on various chess game-sharing platforms.

### Conclusion

The ability to generate a full chess layout from an image is something that can be very useful to professionals and other people trying to play chess. Mainly in the case of someone wanting to recreate a board layout from a video or livestream, this tool can be more efficient than recreating a board state manually. Our model was able to do this with some high degree of accuracy, though it is not perfect and may have some issues generalizing to images of layouts which use a different style or skin. This idea can be greatly improved in the future, potentially by adding the ability to recreate a board from a picture taken of a real chess board, though this would likely require a much more complicated model.


# Footnotes
<a name="wikipedia"></a>1.[^](#notation): “Chess Notation.” *Wikipedia*, Wikimedia Foundation, 27 Feb. 2022, https://en.wikipedia.org/wiki/Chess_notation. <br> 
<a name="livechess"></a>2.[^](#livechess): Quintana, David Mallasén, Alberto Antonio Del Barrio García, και Manuel Prieto-Matías. ‘LiveChess2FEN: a Framework for Classifying Chess Pieces based on CNNs’. CoRR abs/2012.06858 (2020): n. pag. https://arxiv.org/pdf/2012.06858v1.pdf.<br>
<a name="livechess"></a>3.[^](#imagenet): “Image Classification on ImageNet.” Papers With Code, https://paperswithcode.com/sota/image-classification-on-imagenet?tag_filter=17. Accessed 10 June 2022.<br>
